In [ ]:
!pip install google-cloud-aiplatform
!pip install langchain
!pip install chromadb
!pip install pytube
!pip install youtube-transcript-api
!pip install gradio
from google.cloud import aiplatform

In [2]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [3]:
import vertexai
PROJECT_ID = "ai-projects-404911"
vertexai.init(project=PROJECT_ID)

In [4]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI

In [6]:
llm = VertexAI(
model_name="text-bison@001",
max_output_tokens=256,
temperature=0.1, #ouput
top_p=0.8,
top_k=40,
verbose=True,
)

# Top p - Set the probability (minimum) that a token must qualify in order to be picked as a next possible token
# Top k - Number of tokens to consider out of which you want to pick the next possible token from.

In [7]:
from langchain.embeddings import VertexAIEmbeddings

# Embedding
EMBEDDING_QPM = 100  #thresholds
EMBEDDING_NUM_BATCH =5 #at diff batches
embeddings = VertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

# Batch predictions are a way to efficiently send large numbers of  text prompts requests.
# Different from online prediction, where you are limited to one input request at a time,
# you can send a large number of LLM requests in a single batch request

In [8]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=A8jyW_6hCGU&t=161s", add_video_info=True)
result = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_documents(result)
print(f"# of documents = {len(docs)}")

# of documents = 12


In [10]:
#convert text to vector to Cromadb
db = Chroma.from_documents(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

In [12]:
# Create a retriever chain to answer the question. This is where we associate the Vertex AI Text Bison model LLM and the retriever that retrieves the embeddings from Chroma DB.
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [14]:
def sm_ask(question, print_results=True):
  video_subset = qa({"query": question})
  context = video_subset
  prompt = f"""
  Answer the following question in a detailed manner, using information from the text below. If the answer is not in the text,say I dont know and do not generate your own response.

  Question:
  {question}
  Text:
  {context}

  Question:
  {question}

  Answer:
  """
  parameters = {
  "temperature": 0.1,
  "max_output_tokens": 256,
  "top_p": 0.8,
  "top_k": 40
  }
  response = llm.predict(prompt, **parameters)
  return {
  "answer": response

  }

  # Define your prompt to ask questions and get answers from the indexed content.

In [15]:
# Integrate the LLM application with Gradio for a visual front end interaction.
import gradio as gr
def get_response(input_text):
  response = sm_ask(input_text)
  return response

grapp = gr.Interface(fn=get_response, inputs="text", outputs="text")
grapp.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://74f17e56119a6668f8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
